<a href="https://colab.research.google.com/github/FranciscoBPereira/Bioinformatica25-26/blob/main/Bioinfo2526_Aula9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Instalar Biopython**
https://biopython.org/


In [ ]:
!pip install BioPython

from Bio.PDB import *


**Tarefa 1: Acesso à PDB para obtenção de um ficheiro cif**

**1.1 Obter informação sobre a estrutura 3D da proteína com id 1JM7**

In [ ]:
# Obter o ficheiro cif a partir da base de dados PDB

pdb = PDBList()
pdb.retrieve_pdb_file('1JM7', file_format='mmCif')

**1.2 Criar e trabalhar com um objeto Structure**

In [ ]:
# Criar um objeto Structure a partir da informação do ficheiro cif (ignorar os warnings)

parser = MMCIFParser()
estrutura = parser.get_structure('1JM7','jm/1jm7.cif')

In [ ]:
# Verifica o modo experimental usado para calcular a estrutura tridimensional

print('Metodo: ', estrutura.header['structure_method'])

In [ ]:
# Verificar quantos modelos existem no ficheiro

for modelo in estrutura:
  print('Modelo:',modelo.id)

**Questão:**

Explique o motivo pelo qual existem vários modelos?

In [ ]:
# Selecionar apenas o primeiro modelo
# Verificar quantas cadeias tem este modelo

model1 = estrutura[0]
for cadeia in model1:
  print('L Cadeia:',cadeia.id)

In [ ]:
# Apresentar informação mais detalhada do modelo
# Cadeias
# Resíduos por cadeia

for cadeia in model1:
  print('Cadeia:',cadeia.id)
  for residuo in cadeia:
      print('\t\tResiduo:',residuo.resname,"(",residuo.id[1],")")


In [ ]:
# Apresentar informação completa do modelo:
# Cadeias
# Resíduos por cadeia
# Átomos por resíduo, com localização

for cadeia in model1:
  print('Cadeia:',cadeia.id)
  for residuo in cadeia:
      print('\t\tResiduo:',residuo.resname,"(",residuo.id[1],")")
      for atomo in residuo:
        print("\t\t\tAtomo:",atomo.name,"\
        \tCoordenadas: (X:",atomo.coord[0],\
        "L Y:",atomo.coord[1],\
        "L Z:",atomo.coord[2],")")

In [ ]:
# Contar o número de átomos por cadeia

for cadeia in model1:
  print('Cadeia:',cadeia.id)
  num_atomos = 0
  for residuo in cadeia:
      for atomo in residuo:
        num_atomos += 1
  print('\t\tNúmero de átomos:',num_atomos)



In [ ]:
# Contar o número de átomos por resíduo

# Completar #


In [ ]:
# Apresentar uma figura 3D com a localização de todos os átomos deste modelo

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16,9))
ax3d=fig.add_subplot(111,projection='3d')

for atomo in model1.get_atoms():
  ax3d.scatter(atomo.coord[0],atomo.coord[1],atomo.coord[2])

ax3d.set_xlabel('X')
ax3d.set_ylabel('Y')
ax3d.set_zlabel('Z')
plt.show()

**Tarefa 2 - Ligação POST com API da PDB para obtenção de informação**

In [ ]:
import requests
import pandas as pd

In [ ]:
# Efetuar um POST request à API da PDB
# https://data.rcsb.org/#data-api

# Un ficheiro json especifica os detalhes do pedido
# Neste caso pretende-se identificar entradas que satisfaçam o seguinte critério: 'Severe acute respiratory syndrome coronavirus 2'

query_url = 'https://search.rcsb.org/rcsbsearch/v2/query'

query = {
    'query': {
        "type": 'terminal',
        'service': 'text',
        'parameters': {
            'attribute': 'rcsb_entity_source_organism.taxonomy_lineage.name',
            'operator': 'exact_match',
            'value': 'Severe acute respiratory syndrome coronavirus 2'
        }
    },
    'return_type': 'entry'
}

response = requests.post(query_url, json=query)
response.raise_for_status()
results = response.json()

pdb_ids = [r['identifier'] for r in results.get('result_set', [])]
print(f'Foram encontradas {len(pdb_ids)} estruturas SARS-CoV-2 na PDB.')
print(pdb_ids)


In [ ]:

# Obter alguma informação sobre as entradas identificadas

structures = []

for pdb_id in pdb_ids[:20]:  # limitar aos 20 primeiros para exemplo
    url = f'https://data.rcsb.org/rest/v1/core/entry/{pdb_id}'
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        info = {
            'PDB_ID': pdb_id,
            'Título': data.get('struct', {}).get('title'),
            'Método': data.get("exptl", [{}])[0].get("method", ""),
            'Resolução (Å)': data.get('rcsb_entry_info', {}).get('resolution_combined', [None])[0],
            'Data de revisao': data.get('rcsb_accession_info', {}).get('revision_date', ''),
        }
        structures.append(info)

for structure in structures:
    print(structure)

In [ ]:
# Criar um dataframe com a informação recolhida

df = pd.DataFrame(structures)
df['Ano'] = pd.to_datetime(df['Data de revisao']).dt.year
print(df.head())

In [ ]:
# Guardar a informação do dataframe num ficheiro csv

# Guardar como CSV
df.to_csv('sarscov2_structures_summary.csv', index=False)
print('\n Ficheiro criado com sucesso!')

In [ ]:
# Caso seja relevante é possível obter o ficheiro detalhado de uma das entradas identificadas

idP = pdb_ids[0]
print(idP)


pdbl = PDBList()
file = pdbl.retrieve_pdb_file(idP, file_format='mmCif')

print(f' Estrutura {pdb_ids[0]} descarregada em: {file}')



In [ ]:
# Quantos modelos tem o ficheiro desta proteína?
# Quantas cadeias tem cada modelo?

# Completar #


In [ ]:
# Visualização de gráficos sobre a informação armazenada

import plotly.express as px

# Distribuição por método experimental
fig1 = px.histogram(
    df, x='Método', color='Método',
    title='Métodos experimentais usados nas estruturas SARS-CoV-2',
    text_auto=True
)
fig1.update_layout(xaxis_title='Método', yaxis_title='Número de estruturas')
fig1.show()